In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import time
import os
import copy
import shutil# **Testing**

In [2]:
len(os.listdir('/kaggle/input/stanford-dogs-dataset/images/Images'))

120

In [3]:
basePath = '/kaggle/'

modelSave = basePath+'working/weights.pth'
data_dir = basePath+'input/stanford-dogs-dataset/images/Images/'
working_dir = basePath+'working/'
model_file = '/kaggle/input/alexnet/alexnet.pth'

num_classes = 120
batch_size = 8
num_epochs = 50
input_size = 224

In [4]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs,labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [6]:
def set_parameter_requires_grad(model):
    for param in model.parameters():
        param.requires_grad = False

In [7]:
def initialize_model(num_classes):
    alexnet = models.alexnet(pretrained=True)
    set_parameter_requires_grad(alexnet)
    num_ftrs = alexnet.classifier[6].in_features

    alexnet.classifier[6] = nn.Linear(num_ftrs,1024)
    model = nn.Sequential( alexnet,nn.Linear(1024,num_classes), nn.Softmax(dim = 1))
#     alexnet.classifier[6] = nn.Linear(num_ftrs,2048)
#     model = nn.Sequential( alexnet,
#                           nn.ReLU(),
#                           nn.BatchNorm1d(2048),
#                           nn.Linear(2048,1024),
#                           nn.ReLU(),
#                           nn.BatchNorm1d(1024),
#                           nn.Linear(1024,num_classes), 
#                           nn.Softmax(dim = 1))

    return model

In [8]:
# def initialize_model(num_classes):
#     alexnet = models.alexnet(pretrained=True)
#     set_parameter_requires_grad(alexnet)
#     num_ftrs = alexnet.classifier[6].in_features   

#     alexnet.classifier[6] = nn.Linear(num_ftrs,4096)
#     model = nn.Sequential( alexnet,
#                           nn.ReLU(),
#                           nn.BatchNorm1d(4096),
#                           nn.Linear(4096,1024),
#                           nn.ReLU(),
#                           nn.BatchNorm1d(1024),
#                           nn.Linear(1024,num_classes), 
#                           nn.Softmax(dim = 1))

#     return model

In [9]:
def getTrainDataLoaders():
    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

    print("Initializing Datasets and Dataloaders...")

    # Create training and validation datasets
    image_datasets = {x: datasets.ImageFolder(os.path.join(working_dir, x), data_transforms[x]) for x in ['train', 'val']}
    # Create training and validation dataloaders
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

    return dataloaders_dict

In [10]:
def getUpdatablePara(model):
    params_to_update = model.parameters()
    print("Params to learn:")
    params_to_update = []
    for name,param in model.named_parameters():
        if  param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
    return params_to_update

In [11]:
def split_train_test_data(test_size = 0.75):
    for dog_class in os.listdir(data_dir):
        print(dog_class)
        class_path = os.path.join(data_dir, dog_class)
        img_name_list = os.listdir(class_path)
        train_list = img_name_list[:int(test_size * len(img_name_list))]
        val_list = img_name_list[int(test_size * len(img_name_list)):]
        
        destination_folder = os.path.join(working_dir, 'train', dog_class)
        os.makedirs(destination_folder)            
        for img in train_list:
            source = os.path.join(class_path, img)
            destination = os.path.join(destination_folder, img)
            dest = shutil.copyfile(source, destination) 
        
        destination_folder = os.path.join(working_dir, 'val', dog_class)
        os.makedirs(destination_folder)
        for img in val_list:
            source = os.path.join(class_path, img)            
            destination = os.path.join(destination_folder, img)
            dest = shutil.copyfile(source, destination) 

In [12]:
def copy_model_to_cache():
    cache_dir = os.path.expanduser(os.path.join('~', '.torch'))
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    models_dir = os.path.join(cache_dir, 'models')
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)
    
    dest = shutil.copyfile(model_file, os.path.join(models_dir, os.path.basename(model_file)))

# **Prepare Train and Test Data**

In [13]:
split_train_test_data(0.8)

n02109961-Eskimo_dog
n02094258-Norwich_terrier
n02090721-Irish_wolfhound
n02108089-boxer
n02096437-Dandie_Dinmont
n02106382-Bouvier_des_Flandres
n02102480-Sussex_spaniel
n02089078-black-and-tan_coonhound
n02107312-miniature_pinscher
n02085620-Chihuahua
n02110185-Siberian_husky
n02093428-American_Staffordshire_terrier
n02111277-Newfoundland
n02098413-Lhasa
n02115913-dhole
n02095570-Lakeland_terrier
n02100735-English_setter
n02116738-African_hunting_dog
n02106166-Border_collie
n02093991-Irish_terrier
n02097658-silky_terrier
n02091635-otterhound
n02091831-Saluki
n02110063-malamute
n02112706-Brabancon_griffon
n02110627-affenpinscher
n02112137-chow
n02105056-groenendael
n02092339-Weimaraner
n02090622-borzoi
n02089973-English_foxhound
n02099712-Labrador_retriever
n02107574-Greater_Swiss_Mountain_dog
n02113186-Cardigan
n02105251-briard
n02093859-Kerry_blue_terrier
n02097209-standard_schnauzer
n02092002-Scottish_deerhound
n02087394-Rhodesian_ridgeback
n02088466-bloodhound
n02088632-bluetick
n0

# **Training and Validation**

In [14]:
alexnet = initialize_model(num_classes)
alexnet = alexnet.to(device)
#print(alexnet)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth


In [15]:
dataloaders = getTrainDataLoaders()

Initializing Datasets and Dataloaders...


In [16]:
params_to_update = getUpdatablePara(alexnet)

Params to learn:
	 0.classifier.6.weight
	 0.classifier.6.bias
	 1.weight
	 1.bias


In [17]:
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
# optimizer_ft = optim.Adam(params_to_update, lr=0.001)

#loss function 
criterion = nn.CrossEntropyLoss()

alexnet, hist = train_model(alexnet, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

torch.save(alexnet.state_dict(), modelSave)

Epoch 0/49
----------
train Loss: 4.7790 Acc: 0.0329
val Loss: 4.7419 Acc: 0.0702

Epoch 1/49
----------
train Loss: 4.7137 Acc: 0.1041
val Loss: 4.6570 Acc: 0.1574

Epoch 2/49
----------
train Loss: 4.6699 Acc: 0.1457
val Loss: 4.6339 Acc: 0.1749

Epoch 3/49
----------
train Loss: 4.6533 Acc: 0.1584
val Loss: 4.6260 Acc: 0.1824

Epoch 4/49
----------
train Loss: 4.6489 Acc: 0.1604
val Loss: 4.6252 Acc: 0.1804

Epoch 5/49
----------
train Loss: 4.6451 Acc: 0.1626
val Loss: 4.6230 Acc: 0.1824

Epoch 6/49
----------
train Loss: 4.6446 Acc: 0.1635
val Loss: 4.6217 Acc: 0.1814

Epoch 7/49
----------
train Loss: 4.6418 Acc: 0.1644
val Loss: 4.6215 Acc: 0.1824

Epoch 8/49
----------
train Loss: 4.6396 Acc: 0.1662
val Loss: 4.6212 Acc: 0.1826

Epoch 9/49
----------
train Loss: 4.6398 Acc: 0.1661
val Loss: 4.6200 Acc: 0.1836

Epoch 10/49
----------
train Loss: 4.6394 Acc: 0.1656
val Loss: 4.6211 Acc: 0.1838

Epoch 11/49
----------
train Loss: 4.6373 Acc: 0.1692
val Loss: 4.6198 Acc: 0.1819

Ep